In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/homecreditprocessed/train_application_bureau_with_label.csv
/kaggle/input/homecreditprocessed/test_application_bureau_drop_0importance.csv
/kaggle/input/homecreditprocessed/test_bureau.csv
/kaggle/input/homecreditprocessed/train_bureau2.csv
/kaggle/input/homecreditprocessed/train_application_bureau.csv
/kaggle/input/homecreditprocessed/test_application_previous_without_collinearity.csv
/kaggle/input/homecreditprocessed/train_application_bureau_drop_0importance.csv
/kaggle/input/homecreditprocessed/train_application_previous_dropped_collinearity.csv
/kaggle/input/home-credit-default-risk/sample_submission.csv
/kaggle/input/home-credit-default-risk/credit_card_balance.csv
/kaggle/input/home-credit-default-risk/installments_payments.csv
/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv
/kaggle/input/home-credit-default-risk/previous_application.csv
/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv
/kaggle/input/home-credit-default-risk/burea

In [3]:
# modeling 
import lightgbm as lgb

# utilities
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 22
import seaborn as sns
# memory management
import gc

In [20]:
train_application = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
train_bureau = pd.read_csv('../input/homecreditprocessed/train_bureau2.csv')

#train_previous = pd.read_csv('../input/homecreditprocessed/train_previous_processed_2.csv')

test_application_bureau = pd.read_csv('../input/homecreditprocessed/test_bureau.csv')
#test_application_previous = pd.read_csv('../input/homecreditprocessed/test_previous.csv')

# Bureau remove collinearity

In [22]:
train_application_bureau = train_application.merge(train_bureau, on='SK_ID_CURR', how = 'left')
train_application_bureau_label = train_application_bureau['TARGET']
train_application_bureau = train_application_bureau.drop(['TARGET'],axis=1)

In [ ]:
#train_application_bureau_with_label = train_application_bureau.to_csv('train_application_bureau_with_label.csv',index=False)

In [8]:
train_application_bureau.head()

SK_ID_CURR  TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
0      100002       1         Cash loans           M            N   
1      100003       0         Cash loans           F            N   
2      100004       0    Revolving loans           M            Y   
3      100006       0         Cash loans           F            N   
4      100007       0         Cash loans           M            N   

  FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  \
0               Y             0          202500.0    406597.5      24700.5   
1               N             0          270000.0   1293502.5      35698.5   
2               Y             0           67500.0    135000.0       6750.0   
3               Y             0          135000.0    312682.5      29686.5   
4               Y             0          121500.0    513000.0      21865.5   

   ...  client_bureau_balance_STATUS_X_count_max  \
0  ...                                       3.0   
1  ...                                       NaN   
2  ...                                       NaN   
3  ...                                       NaN   
4  ...                                       NaN   

  client_bureau_balance_STATUS_0_count_max  \
0                                     18.0   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

  client_bureau_balance_STATUS_C_count_mean  \
0                                     2.875   
1                                       NaN   
2                                       NaN   
3                                       NaN   
4                                       NaN   

  client_bureau_balance_MONTHS_BALANCE_count_mean  \
0                                           13.75   
1                                             NaN   
2                                             NaN   
3                                             NaN   
4                                             NaN   

  client_bureau_balance_STATUS_X_count_sum  \
0                                     15.0   
1                                      0.0   
2                                      0.0   
3                                      NaN   
4                                      0.0   

  client_bureau_balance_STATUS_0_count_sum  \
0                                     45.0   
1                                      0.0   
2                                      0.0   
3                                      NaN   
4                                      0.0   

   client_bureau_balance_STATUS_C_count_max  \
0                                      13.0   
1                                       NaN   
2                                       NaN   
3                                       NaN   
4                                       NaN   

   client_bureau_balance_MONTHS_BALANCE_count_max  \
0                                            22.0   
1                                             NaN   
2                                             NaN   
3                                             NaN   
4                                             NaN   

   client_bureau_balance_STATUS_C_count_sum  \
0                                      23.0   
1                                       0.0   
2                                       0.0   
3                                       NaN   
4                                       0.0   

   client_bureau_balance_MONTHS_BALANCE_count_sum  
0                                           110.0  
1                                             0.0  
2                                             0.0  
3                                             NaN  
4                                             0.0  

[5 rows x 309 columns]

In [9]:
train_application_bureau.to_csv('train_application_bureau.csv',index=False)

In [29]:
test_application_bureau.head()

SK_ID_CURR NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  \
0      100001         Cash loans           F            N               Y   
1      100005         Cash loans           M            N               Y   
2      100013         Cash loans           M            Y               Y   
3      100028         Cash loans           F            N               Y   
4      100038         Cash loans           M            Y               N   

   CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  AMT_GOODS_PRICE  \
0             0          135000.0    568800.0      20560.5         450000.0   
1             0           99000.0    222768.0      17370.0         180000.0   
2             0          202500.0    663264.0      69777.0         630000.0   
3             2          315000.0   1575000.0      49018.5        1575000.0   
4             1          180000.0    625500.0      32067.0         625500.0   

   ... client_bureau_balance_STATUS_X_count_max  \
0  ...                                      9.0   
1  ...                                      1.0   
2  ...                                     40.0   
3  ...                                     60.0   
4  ...                                      NaN   

  client_bureau_balance_STATUS_0_count_max  \
0                                     12.0   
1                                      7.0   
2                                     34.0   
3                                     34.0   
4                                      NaN   

  client_bureau_balance_STATUS_C_count_mean  \
0                                 15.714286   
1                                  1.666667   
2                                 25.750000   
3                                 21.916667   
4                                       NaN   

  client_bureau_balance_MONTHS_BALANCE_count_mean  \
0                                       24.571429   
1                                        7.000000   
2                                       57.500000   
3                                       46.666667   
4                                             NaN   

  client_bureau_balance_STATUS_X_count_sum  \
0                                     30.0   
1                                      2.0   
2                                     41.0   
3                                    133.0   
4                                      NaN   

   client_bureau_balance_STATUS_0_count_sum  \
0                                      31.0   
1                                      14.0   
2                                      79.0   
3                                     164.0   
4                                       NaN   

   client_bureau_balance_STATUS_C_count_max  \
0                                      44.0   
1                                       5.0   
2                                      44.0   
3                                      62.0   
4                                       NaN   

   client_bureau_balance_MONTHS_BALANCE_count_max  \
0                                            52.0   
1                                            13.0   
2                                            69.0   
3                                            70.0   
4                                             NaN   

   client_bureau_balance_STATUS_C_count_sum  \
0                                     110.0   
1                                       5.0   
2                                     103.0   
3                                     263.0   
4                                       NaN   

   client_bureau_balance_MONTHS_BALANCE_count_sum  
0                                           172.0  
1                                            21.0  
2                                           230.0  
3                                           560.0  
4                                             NaN  

[5 rows x 308 columns]

In [30]:
train_application_bureau = pd.get_dummies(train_application_bureau)
#train_application_bureau.head()

In [ ]:
# Threshold for removing correlated variables
threshold = 0.9

# Absolute value correlation matrix
corr_matrix = train_application_bureau.corr().abs()
corr_matrix.head()

In [ ]:
# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.head()

In [ ]:
# Select columns with correlations above threshold
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

print('There are %d columns to remove.' % (len(to_drop)))

In [ ]:
train_application_bureau_dropped = train_application_bureau.drop(columns = to_drop)
test_application_bureau_dropped = test_application_bureau.drop(columns = to_drop)

In [ ]:
train_application_bureau_dropped.to_csv('train_application_bureau_dropped_collinear.csv',index=False)
test_application_bureau_dropped.to_csv('test_application_bureau_dropped_collinear.csv',index=False)

In [ ]:
train_application_bureau_dropped = train_application_bureau_dropped.drop(columns = ['SK_ID_CURR'])
test_application_bureau_dropped = test_application_bureau_dropped.drop(columns = ['SK_ID_CURR'])

In [ ]:
train = pd.get_dummies(train_application_bureau_dropped)
test = pd.get_dummies(test_application_bureau_dropped)

In [ ]:
# Match the columns in the dataframes
train, test = train.align(test, join = 'inner', axis = 1)
print('Training shape: ', train.shape)
print('Testing shape: ', test.shape)

In [ ]:
submission_remove_colliear, feature_importances, metrics = model(train, test)

## feature importance

In [28]:
# Initialize an empty array to hold feature importances
feature_importances = np.zeros(train.shape[1])

# Create the model with several hyperparameters
model = lgb.LGBMClassifier(objective='binary', boosting_type = 'goss', n_estimators = 10000, class_weight = 'balanced')

NameError: name 'train' is not defined

In [ ]:
import re
train_re = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test_re = test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [ ]:
# Fit the model twice to avoid overfitting
for i in range(2):
    
    # Split into training and validation set
    train_features, valid_features, train_y, valid_y = train_test_split(train_re, train_application_bureau_label, test_size = 0.25, random_state = i)
    
    # Train using early stopping
    model.fit(train_features, train_y, early_stopping_rounds=100, eval_set = [(valid_features, valid_y)], 
              eval_metric = 'auc', verbose = 200)
    
    # Record the feature importances
    feature_importances += model.feature_importances_

In [ ]:
# Make sure to average feature importances! 
feature_importances = feature_importances / 2
feature_importances = pd.DataFrame({'feature': list(train_re.columns), 'importance': feature_importances}).sort_values('importance', ascending = False)

In [ ]:
# Find the features with zero importance
zero_features = list(feature_importances[feature_importances['importance'] == 0.0]['feature'])
print('There are %d features with 0.0 importance' % len(zero_features))
feature_importances.tail()

In [ ]:
train_re = train_re.drop(columns = zero_features)
test_re = test_re.drop(columns = zero_features)


In [ ]:
print('Training shape: ', train_re.shape)
print('Testing shape: ', test_re.shape)

In [ ]:
train_re.to_csv('train_application_bureau_drop_0importance.csv',index=False)
test_re.to_csv('test_application_bureau_drop_0importance.csv',index=False)

In [16]:
def plot_feature_importances(df, threshold = 0.9):
    
    plt.rcParams['font.size'] = 18
    
    # Sort features according to importance
    df = df.sort_values('importance', ascending = False).reset_index()
    
    # Normalize the feature importances to add up to one
    df['importance_normalized'] = df['importance'] / df['importance'].sum()
    df['cumulative_importance'] = np.cumsum(df['importance_normalized'])

    # Make a horizontal bar chart of feature importances
    plt.figure(figsize = (10, 6))
    ax = plt.subplot()
    
    # Need to reverse the index to plot most important on top
    ax.barh(list(reversed(list(df.index[:15]))), 
            df['importance_normalized'].head(15), 
            align = 'center', edgecolor = 'k')
    
    # Set the yticks and labels
    ax.set_yticks(list(reversed(list(df.index[:15]))))
    ax.set_yticklabels(df['feature'].head(15))
    
    # Plot labeling
    plt.xlabel('Normalized Importance'); plt.title('Feature Importances')
    plt.show()
    
    # Cumulative importance plot
    plt.figure(figsize = (8, 6))
    plt.plot(list(range(len(df))), df['cumulative_importance'], 'r-')
    plt.xlabel('Number of Features'); plt.ylabel('Cumulative Importance'); 
    plt.title('Cumulative Feature Importance');
    plt.show();
    
    importance_index = np.min(np.where(df['cumulative_importance'] > threshold))
    print('%d features required for %0.2f of cumulative importance' % (importance_index + 1, threshold))
    
    return df

In [ ]:
norm_feature_importances = plot_feature_importances(feature_importances)

In [17]:
def model(features, test_features, encoding = 'ohe', n_folds = 5):
 
    
    # Extract the ids
    train_ids = features['SK_ID_CURR']
    test_ids = test_features['SK_ID_CURR']
    
    # Extract the labels for training
    labels = features['TARGET']
    
    # Remove the ids and target
    features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
    
    # One Hot Encoding
    if encoding == 'ohe':
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)
        
        # Align the dataframes by the columns
        features, test_features = features.align(test_features, join = 'inner', axis = 1)
        
        # No categorical indices to record
        cat_indices = 'auto'
    
    # Integer label encoding
    elif encoding == 'le':
        
        # Create a label encoder
        label_encoder = LabelEncoder()
        
        # List for storing categorical indices
        cat_indices = []
        
        # Iterate through each column
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                # Map the categorical features to integers
                features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))

                # Record the categorical indices
                cat_indices.append(i)
    
    # Catch error if label encoding scheme is not valid
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = list(features.columns)
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = False, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                                   class_weight = 'balanced', learning_rate = 0.05, 
                                   reg_alpha = 0.1, reg_lambda = 0.1, 
                                   subsample = 0.8, n_jobs = -1, random_state = 50)
        
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, feature_importances, metrics

In [ ]:
train_re['TARGET'] = train_application_bureau_label
train_re['SK_ID_CURR'] = train_application_bureau['SK_ID_CURR']
test_re['SK_ID_CURR'] = test_application_bureau['SK_ID_CURR']

In [ ]:
submission, feature_importances, metrics = model(train_re, test_re)

In [ ]:
submission.to_csv('application_bureau_submission.csv', index = False)
#public score: 0.75644

In [ ]:
norm_feature_importances = plot_feature_importances(feature_importances)

# Comparison

In [ ]:
train['TARGET'] = train_application_bureau_label
train['SK_ID_CURR'] = train_application_bureau['SK_ID_CURR']
test['SK_ID_CURR'] = test_application_bureau['SK_ID_CURR']

In [ ]:
submission2, feature_importances2, metrics = model(train, test)

In [ ]:
submission2.to_csv('application_bureau_submission2.csv', index = False)
#0.75709

In [ ]:
feature_importances2

In [ ]:
# Find the features with zero importance
zero_features = list(feature_importances2[feature_importances2['importance'] == 0.0]['feature'])
print('There are %d features with 0.0 importance' % len(zero_features))
zero_features

In [ ]:
train_2 = train.drop(columns = zero_features)
test_2 = test.drop(columns = zero_features)

In [ ]:
submission3, feature_importances3, metrics = model(train_2, test_2)

In [ ]:
submission3.to_csv('application_bureau_submission3.csv', index = False)
#0.75717